In [ ]:
# RU_BI_LAB_SSIS (source)
RU_BI_LAB_SSIS_server_name = r"localhost"
RU_BI_LAB_SSIS_db_name = r"RU.BI.LAB.SSIS.DB"
# CrimeStatistic_DWH (destination)
CrimeStatistic_DWH_server_name = r"localhost"
CrimeStatistic_DWH_db_name = r"CrimeStatistic.DWH"
undoc_files_path = r"C:\RU.BI.LAB.SSIS\Source files\undoc-changed\csv"
worldbank_files_path = r"C:\RU.BI.LAB.SSIS\Source files\worldbank-changed"
worldbank_file_name = r"1.csv"

In [ ]:
%%HTML
<style type="text/css">
    table.dataframe td, table.dataframe th {
        border-style: solid;
    }
</style>

## **BI.LAB.SSIS.task.02**

**Subtask.02.01. Test** **ExtractCountryRef.dtsx package**

Test.02.01.01. Compare rows count in source db RU.BI.LAB.SSIS to staging.countriesref\_untyped table

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import pyodbc
cnxn_src = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server="+RU_BI_LAB_SSIS_server_name+";"
                      "Database="+RU_BI_LAB_SSIS_db_name+";"
                      "Trusted_Connection=yes;")

cnxn_dst = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server="+CrimeStatistic_DWH_server_name+";"
                      "Database="+CrimeStatistic_DWH_db_name+";"
                      "Trusted_Connection=yes;")                      
df_source = pd.read_sql_query("SELECT count(1) AS [RowCount_source] FROM [dbo].[CountryRef]", cnxn_src)
df_dwh_untyped = pd.read_sql_query("SELECT count(1) AS [RowCount_untyped] FROM [staging].[countriesref_untyped]", cnxn_dst)
df_result = df_source.join(df_dwh_untyped)
df_result['TestResult'] = np.where(df_result['RowCount_source'] == df_result['RowCount_untyped'], 'SUCCESS', 'FAILED')
df_result.index += 1
df_result

## **BI.LAB.SSIS.task.04**

**Subtask.04.01.** **Test Load\_Staging\_UNCrime.dtsx package**

Test.04.01.01. Compare rows count in source UNCrime files to staging.uncrime\_untyped

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import pyodbc
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server="+CrimeStatistic_DWH_server_name+";"
                      "Database="+CrimeStatistic_DWH_db_name+";"
                      "Trusted_Connection=yes;")
df_dwh_untyped = pd.read_sql_query("SELECT [FileName], count(1) as [RowCounts_uncrime_untyped] FROM staging.uncrime_untyped GROUP BY [FileName] ORDER BY [FileName]", cnxn)
df_csv = pd.DataFrame({'FileName': [], 'RowCounts_file': pd.Series([], dtype=np.dtype("int32"))})
onlyfiles = [f for f in listdir(undoc_files_path) if isfile(join(undoc_files_path, f))]
for f in onlyfiles:
    df = pd.read_csv(undoc_files_path + "\\" + f)
    df_csv = df_csv.append({'FileName': f, 'RowCounts_file': len(df.index)-1}, ignore_index = True) 
df_result = df_csv.join(df_dwh_untyped.set_index('FileName'), on='FileName')
df_result['TestResult'] = np.where(df_result['RowCounts_file'] == df_result['RowCounts_uncrime_untyped'], 'SUCCESS', 'FAILED')
df_result.index += 1
df_result

## **BI.LAB.SSIS.task.05**

**Subtask.05.01. Test Load\_staging\_WorldBank.dtsx package**

Test.05.01.01. Compare rows count in source WorldBank file to staging.population\_untyped

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import pyodbc
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server="+CrimeStatistic_DWH_server_name+";"
                      "Database="+CrimeStatistic_DWH_db_name+";"
                      "Trusted_Connection=yes;")
df_dwh_untyped = pd.read_sql_query("SELECT [FileName], count(1) as [RowCounts_population_untyped] FROM staging.population_untyped GROUP BY [FileName] ORDER BY [FileName]", cnxn)
df_csv = pd.DataFrame({'FileName': [], 'RowCounts_file': pd.Series([], dtype=np.dtype("int32"))})
wb_files_path = worldbank_files_path
file_name = worldbank_file_name
df = pd.read_csv(wb_files_path + "\\" + file_name, skiprows=3)
df_csv = df_csv.append({'FileName': file_name, 'RowCounts_file': len(df.index)-1}, ignore_index = True) 
df_result = df_csv.join(df_dwh_untyped.set_index('FileName'), on='FileName')
df_result['TestResult'] = np.where(df_result['RowCounts_file'] == df_result['RowCounts_population_untyped'], 'SUCCESS', 'FAILED')
df_result.index += 1
df_result

## **BI.LAB.SSIS.task.06**

**Subtask.06.01.** **Test Transform\_staging\_CountryRef.dtsx package**

Test.06.01.01. Compare rows count in \[staging\].\[countriesref\_untyped\] to staging.CountryRef

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import pyodbc
cnxn_dst = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server="+CrimeStatistic_DWH_server_name+";"
                      "Database="+CrimeStatistic_DWH_db_name+";"
                      "Trusted_Connection=yes;")                      
df_dwh_untyped = pd.read_sql_query("SELECT count(1) AS [RowCount_untyped] FROM [staging].[countriesref_untyped]", cnxn_dst)
df_dwh_countryref = pd.read_sql_query("SELECT count(1) AS [RowCount_CountryRef] FROM [staging].[CountryRef]", cnxn_dst)
df_result = df_dwh_untyped.join(df_dwh_countryref)
df_result['TestResult'] = np.where(df_result['RowCount_untyped'] == df_result['RowCount_CountryRef'], 'SUCCESS', 'FAILED')
df_result.index += 1
df_result

**Subtask.06.02. Test Transform\_staging\_Population.dtsx package**

Test.06.02.01. Compare row count in staging.population\_untyped to staging.population\_typed

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import pyodbc
cnxn_dst = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server="+CrimeStatistic_DWH_server_name+";"
                      "Database="+CrimeStatistic_DWH_db_name+";"
                      "Trusted_Connection=yes;")   
df_result = pd.read_sql_query("""WITH pu AS (
SELECT [FileName], count(1) as [RowCount]
FROM staging.population_untyped
GROUP BY [FileName]),
pt AS (
SELECT [FileName], count(1) as [RowCount]
FROM staging.population_typed
GROUP BY [FileName])
SELECT pu.[FileName]
	 , pu.[RowCount] AS [RowCount_untyped]
	 , pt.[RowCount] AS [RowCount_typed]
	 , CASE WHEN pu.[RowCount] = pt.[RowCount] THEN N'SUCCESS' ELSE N'FAILED' END AS [TestResult]
FROM pu
INNER JOIN pt on pt.[FileName] = pu.[FileName]""", cnxn_dst)
df_result


Test.06.02.02. Compare sum population in staging.population\_untyped to staging.population\_typed

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import pyodbc
cnxn_dst = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server="+CrimeStatistic_DWH_server_name+";"
                      "Database="+CrimeStatistic_DWH_db_name+";"
                      "Trusted_Connection=yes;")   
df_result = pd.read_sql_query("""WITH pt AS(
SELECT ISNULL(SUM([1960]), 0)
      +ISNULL(SUM([1961]), 0)
      +ISNULL(SUM([1962]), 0)
      +ISNULL(SUM([1963]), 0)
      +ISNULL(SUM([1964]), 0)
      +ISNULL(SUM([1965]), 0)
      +ISNULL(SUM([1966]), 0)
      +ISNULL(SUM([1967]), 0)
      +ISNULL(SUM([1968]), 0)
      +ISNULL(SUM([1969]), 0)
      +ISNULL(SUM([1970]), 0)
      +ISNULL(SUM([1971]), 0)
      +ISNULL(SUM([1972]), 0)
      +ISNULL(SUM([1973]), 0)
      +ISNULL(SUM([1974]), 0)
      +ISNULL(SUM([1975]), 0)
      +ISNULL(SUM([1976]), 0)
      +ISNULL(SUM([1977]), 0)
      +ISNULL(SUM([1978]), 0)
      +ISNULL(SUM([1979]), 0)
      +ISNULL(SUM([1980]), 0)
      +ISNULL(SUM([1981]), 0)
      +ISNULL(SUM([1982]), 0)
      +ISNULL(SUM([1983]), 0)
      +ISNULL(SUM([1984]), 0)
      +ISNULL(SUM([1985]), 0)
      +ISNULL(SUM([1986]), 0)
      +ISNULL(SUM([1987]), 0)
      +ISNULL(SUM([1988]), 0)
      +ISNULL(SUM([1989]), 0)
      +ISNULL(SUM([1990]), 0)
      +ISNULL(SUM([1991]), 0)
      +ISNULL(SUM([1992]), 0)
      +ISNULL(SUM([1993]), 0)
      +ISNULL(SUM([1994]), 0)
      +ISNULL(SUM([1995]), 0)
      +ISNULL(SUM([1996]), 0)
      +ISNULL(SUM([1997]), 0)
      +ISNULL(SUM([1998]), 0)
      +ISNULL(SUM([1999]), 0)
      +ISNULL(SUM([2000]), 0)
      +ISNULL(SUM([2001]), 0)
      +ISNULL(SUM([2002]), 0)
      +ISNULL(SUM([2003]), 0)
      +ISNULL(SUM([2004]), 0)
      +ISNULL(SUM([2005]), 0)
      +ISNULL(SUM([2006]), 0)
      +ISNULL(SUM([2007]), 0)
      +ISNULL(SUM([2008]), 0)
      +ISNULL(SUM([2009]), 0)
      +ISNULL(SUM([2010]), 0)
      +ISNULL(SUM([2011]), 0)
      +ISNULL(SUM([2012]), 0)
      +ISNULL(SUM([2013]), 0)
      +ISNULL(SUM([2014]), 0)
      +ISNULL(SUM([2015]), 0)
      +ISNULL(SUM([2016]), 0)
      +ISNULL(SUM([2017]), 0)
      +ISNULL(SUM([2018]), 0)
      +ISNULL(SUM([2019]), 0) AS [SumPopulation]
  FROM [staging].[population_typed]),
pu AS (
SELECT ISNULL(SUM([Population]), 0) AS [SumPopulation] 
FROM [staging].[population_unpivoted])
SELECT pt.[SumPopulation] AS [SumPopulation_typed]
     , pu.[SumPopulation] AS [SumPopulation_unpivoted]
	 , CASE WHEN pt.[SumPopulation] = pu.[SumPopulation] THEN 'SUCCESS' ELSE 'FAILED' END AS [TestResult]
  FROM  pt, pu""", cnxn_dst)
df_result

**Subtask.06.03. Test Transform\_staging\_UNCrime.dtsx package**

Test.06.03.01. Compare rows count in \[staging\].\[uncrime\_untyped\] to \[staging\].\[uncrime\_typed\]

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import pyodbc
cnxn_dst = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server="+CrimeStatistic_DWH_server_name+";"
                      "Database="+CrimeStatistic_DWH_db_name+";"
                      "Trusted_Connection=yes;")   
df_result = pd.read_sql_query("""WITH pu AS (
SELECT [FileName], count(1) as [RowCount]
FROM staging.uncrime_untyped
GROUP BY [FileName]),
pt AS (
SELECT [FileName], count(1) as [RowCount]
FROM staging.uncrime_typed
GROUP BY [FileName])
SELECT pu.[FileName]
	 , pu.[RowCount] AS [RowCount_untyped]
	 , pt.[RowCount] AS [RowCount_typed]
	 , CASE WHEN pu.[RowCount] = pt.[RowCount] THEN N'SUCCESS' ELSE N'FAILED' END AS [TestResult]
FROM pu
INNER JOIN pt on pt.[FileName] = pu.[FileName]""", cnxn_dst)
df_result

**Subtask.06.04. Test Load\_dwh\_FactPopulation.dtsx package**

Test.06.04.01. Compare sum population in \[staging\].\[population\_unpivoted\] to \[dwh\].\[FactPopulation\]

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import pyodbc
cnxn_dst = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server="+CrimeStatistic_DWH_server_name+";"
                      "Database="+CrimeStatistic_DWH_db_name+";"
                      "Trusted_Connection=yes;")   
df_result = pd.read_sql_query("""WITH pu AS (
	SELECT CountryCode, [Year], ISNULL(SUM([Population]), 0) AS [SumPopulation]
	FROM [staging].[population_unpivoted]
	--NOT COUNTRIES
	WHERE CountryCode NOT IN (
	 'ARB', 'CEB', 'CHI', 'CSS', 'EAP', 'EAR', 'EAS', 'ECA', 'ECS', 'EMU', 'EUU', 'FCS', 'HIC', 'HPC', 'IBD', 'IBT', 'IDA', 'IDB', 'IDX', 'INX', 'LAC', 'LCN', 'LDC', 'LIC', 'LMC', 'LMY',
	 'LTE', 'MEA', 'MIC', 'MNA', 'NAC', 'OED', 'OSS', 'PRE', 'PSS', 'PST', 'SAS', 'SSA', 'SSF', 'SST', 'TEA', 'TEC', 'TLA', 'TMN', 'TSA', 'TSS', 'UMC', 'WLD', 'XKX')
	GROUP BY CountryCode, [Year]
),
fp AS (
	SELECT dg.CountryCode, dd.CalendarYear, ISNULL(SUM(fp2.[Population]), 0) AS [SumPopulation]
	FROM [dwh].[FactPopulation] fp2
	INNER JOIN dwh.DimGeography dg ON dg.DimGeographyKey = fp2.DimGeographyKey
	INNER JOIN dwh.DimDate dd ON dd.DateSK = fp2.DimDateKey
	GROUP BY dg.CountryCode, dd.CalendarYear
)
SELECT SUM(pu.[SumPopulation]) AS [SumPopulation_unpivoted]
	 , SUM(fp.[SumPopulation]) AS [SumPopulation_fact]
	 , CASE WHEN SUM(pu.[SumPopulation]) <> 0 AND SUM(fp.[SumPopulation]) <> 0 AND SUM(pu.[SumPopulation]) = SUM(fp.[SumPopulation]) THEN N'SUCCESS' ELSE N'FAILED' END AS [TestResult]
FROM pu
LEFT JOIN fp ON fp.CountryCode = pu.CountryCode
	AND fp.CalendarYear = pu.[Year]""", cnxn_dst)
df_result

**Subtask.06.05. Test** **Load\_dwh\_FactCrime.dtsx package**

Test.06.05.01. Compare NumberOfCrime in \[staging\].\[uncrime\_unpivoted\] to \[dwh\].\[FactCrime\]

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import pyodbc
cnxn_dst = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server="+CrimeStatistic_DWH_server_name+";"
                      "Database="+CrimeStatistic_DWH_db_name+";"
                      "Trusted_Connection=yes;")   
df_result = pd.read_sql_query("""WITH 
  uu AS (SELECT SUM([Count]) AS [NumberOfCrime]
	     FROM [staging].[uncrime_unpivoted])
, fc AS (SELECT SUM(NumberOfCrime) AS [NumberOfCrime]
		 FROM [dwh].[FactCrime])
SELECT uu.[NumberOfCrime] AS [NumberOfCrime_uncrime_unpivoted]
	 , fc.[NumberOfCrime] AS [NumberOfCrime_FactCrime]
	 , CASE WHEN uu.[NumberOfCrime] > 0 AND fc.[NumberOfCrime] > 0 AND uu.[NumberOfCrime] = fc.[NumberOfCrime] THEN N'SUCCESS' ELSE N'FAILED' END AS [TestResult]
FROM uu, fc""", cnxn_dst)
df_result

Test.06.05.02. Check broken values for DimGeographyKey in \[dwh\].\[FactCrime\]

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import pyodbc
cnxn_dst = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server="+CrimeStatistic_DWH_server_name+";"
                      "Database="+CrimeStatistic_DWH_db_name+";"
                      "Trusted_Connection=yes;")   
df_result = pd.read_sql_query("""WITH t AS (
SELECT N'DimGeographyKey' AS [Key]
	 , SUM(CASE WHEN fc.DimGeographyKey IS NULL OR dg.DimGeographyKey IS NULL THEN 1 ELSE 0 END) AS [NumberOfBrokenValues]
FROM [dwh].[FactCrime] fc
LEFT JOIN [dwh].[DimGeography] dg ON dg.DimGeographyKey = fc.DimGeographyKey)
SELECT t.[Key]
	 , t.[NumberOfBrokenValues]
	 , CASE WHEN t.[NumberOfBrokenValues] = 0 THEN N'SUCCESS' ELSE N'FAILED' END AS [TestResult]
FROM t""", cnxn_dst)
df_result

Test.06.05.03. Check broken values for DimDateKey in \[dwh\].\[FactCrime\]

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import pyodbc
cnxn_dst = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server="+CrimeStatistic_DWH_server_name+";"
                      "Database="+CrimeStatistic_DWH_db_name+";"
                      "Trusted_Connection=yes;")   
df_result = pd.read_sql_query("""WITH t AS (
SELECT N'DimDateKey' AS [Key]
	 , SUM(CASE WHEN fc.DimDateKey IS NULL OR dg.DateSK IS NULL THEN 1 ELSE 0 END) AS [NumberOfBrokenValues]
FROM [dwh].[FactCrime] fc
LEFT JOIN [dwh].[DimDate] dg ON dg.DateSK = fc.DimDateKey)
SELECT t.[Key]
	 , t.[NumberOfBrokenValues]
	 , CASE WHEN t.[NumberOfBrokenValues] = 0 THEN N'SUCCESS' ELSE N'FAILED' END AS [TestResult]
FROM t""", cnxn_dst)
df_result

Test.06.05.04. Check broken values for DimCrimeTypeKey in \[dwh\].\[FactCrime\]

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import pyodbc
cnxn_dst = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server="+CrimeStatistic_DWH_server_name+";"
                      "Database="+CrimeStatistic_DWH_db_name+";"
                      "Trusted_Connection=yes;")   
df_result = pd.read_sql_query("""WITH t AS (
SELECT N'DimCrimeTypeKey' AS [Key]
	 , SUM(CASE WHEN fc.DimCrimeTypeKey IS NULL OR dg.DimCrimeTypeKey IS NULL THEN 1 ELSE 0 END) AS [NumberOfBrokenValues]
FROM [dwh].[FactCrime] fc
LEFT JOIN [dwh].[DimCrimeType] dg ON dg.DimCrimeTypeKey = fc.DimCrimeTypeKey)
SELECT t.[Key]
	 , t.[NumberOfBrokenValues]
	 , CASE WHEN t.[NumberOfBrokenValues] = 0 THEN N'SUCCESS' ELSE N'FAILED' END AS [TestResult]
FROM t""", cnxn_dst)
df_result

**Subtask.06.06. Test** **Load\_dwh\_DimCountry.dtsx** **package**

Test.06.06.01. Compare number of distinct CountryCode in \[staging\].\[CountryRef\] to \[dwh\].\[DimGeography\]

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import pyodbc
cnxn_dst = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server="+CrimeStatistic_DWH_server_name+";"
                      "Database="+CrimeStatistic_DWH_db_name+";"
                      "Trusted_Connection=yes;")   
df_result = pd.read_sql_query("""SELECT cr.[name] AS [CountryName_staging]
	 , cr.alpha_3 AS [CountryCode_staging]
	 , dg.Country AS [CountryName_dwh]
	 , dg.CountryCode AS [CountryCode_dwh]
	 , CASE WHEN dg.CountryCode IS NOT NULL THEN N'SUCCESS' ELSE N'FAILED' END AS [TestResult]
FROM staging.CountryRef cr
LEFT JOIN [dwh].[DimGeography] dg ON dg.CountryCode = cr.alpha_3""", cnxn_dst)
df_result

**Subtask.06.07. Test** **Load\_dwh\_DimCrimeType.dtsx** **package**

Test.06.07.01. Compare number of distinct CrimeType in \[staging\].\[uncrime\_typed\] to \[dwh\].\[DimCrimeType\]

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import pyodbc
cnxn_dst = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server="+CrimeStatistic_DWH_server_name+";"
                      "Database="+CrimeStatistic_DWH_db_name+";"
                      "Trusted_Connection=yes;")   
df_result = pd.read_sql_query("""WITH st AS 
(SELECT DISTINCT REPLACE(LEFT([FileName], CHARINDEX('.csv', [FileName])-1),'_',' ') AS CrimeType
FROM [staging].[uncrime_typed]
UNION ALL
SELECT N'Unknown')
SELECT st.CrimeType  AS [CrimeType_uncrime_typed]
	 , dct.CrimeType AS [CrimeType_DimCrimeType]
	 , CASE WHEN st.CrimeType IS NULL OR dct.CrimeType IS NULL THEN N'FAILED' ELSE N'SUCCESS' END AS [TestResult]
FROM st
FULL OUTER JOIN [dwh].[DimCrimeType] dct ON UPPER(dct.CrimeType) = UPPER(st.CrimeType)
GROUP BY st.CrimeType, dct.CrimeType""", cnxn_dst)
df_result

**Subtask.06.08. Test** **Load\_dwh\_DimDate.dtsx** **package**

Test.06.08.01. Check that \[dwh\].\[DimDate\] is not empty

In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import pyodbc
cnxn_dst = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server="+CrimeStatistic_DWH_server_name+";"
                      "Database="+CrimeStatistic_DWH_db_name+";"
                      "Trusted_Connection=yes;")   
df_result = pd.read_sql_query("""SELECT COUNT(1) AS [Row_count]
	, CASE WHEN COUNT(1) > 1 THEN N'SUCCESS' ELSE N'FAILED' END AS [TestResult] 
FROM [dwh].[DimDate]""", cnxn_dst)
df_result

**Subtask.06.09. Test** **Main\_DWH.dtsx** **package**

**Subtask.06.10. Test** **Main.dtsx (common)** **package**

**Subtask.06.11. Test** **Main\_advanced.dtsx (advanced)** **package**